In [ ]:
# Section 10 from the ZTM course on Udemy (TensorFlow Developer Certificate: Zero to Mastery)

# Download text dataset

Dataset from Kaggle containing Tweets about natural disaster

In [2]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

#unzip data
import zipfile
zip_ref = zipfile.ZipFile('nlp_getting_started.zip', "r")
zip_ref.extractall()
zip_ref.close()

--2024-01-08 17:01:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.207, 142.250.4.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K   724KB/s    in 0.8s    

2024-01-08 17:01:11 (724 KB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [3]:
# load the downloaded CSV files into dataframes
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility and frac=1 to get all the the entire dataframe
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# The test data doesn't have a target column
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# how many examples of each target class are available in the training dataset
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

The dataset is not completely balanced but the distribution is not very skwed.

~57% have target = 0 (not a Tweet about a real disaster)

43% have target = 1 (Tweet about a real disaster)

In [7]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [8]:
import random
# select a random number of examples from the training data to visualize them
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
All I got in this world is my bros I don't wanna see no harm come they way Ima lil upset right now ??

---

Target: 0 (not real disaster)
Text:
@MarianKeyes Rubber Mudslide! Still laughing!

---

Target: 0 (not real disaster)
Text:
It's cold and my head wants to explode.. The joys of working from home - I'm going back to bed / peace out ????

---

Target: 1 (real disaster)
Text:
Governor allows parole for California school bus hijacker who kidnapped 26 children in 1976. http://t.co/hdAhLgrprl http://t.co/Z1s3T77P3L

---

Target: 0 (not real disaster)
Text:
@DavisKawalya I know @Mauryn143 will be saying her final goodbyes to grandpa as seen on news RiP Me? always open to ideas but may ve curfew

---



In [9]:
# Split training dataset into training and validation sets using an 80/20 split
from sklearn.model_selection import train_test_split

# y = target column, while X = text column, all other columns will not be included
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.2, # dedicate 20% of samples to validation set
                                                                            random_state=42)

In [11]:
# Check the shape of the splits
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6090,), (6090,), (1523,), (1523,))

In [12]:
# have a look at the first 10 examples and their corresponding labels
train_sentences[:10], train_labels[:10]

(array(['Detonation fashionable mountaineering electronic watch water-resistant couples leisure tab\x89Û_ http://t.co/E61x9Y65QD http://t.co/OVLET0gDqm',
        "Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/tFan6qq2Ys http://t.co/rAkwWritPo",
        'I-77 Mile Marker 31 to 40 South Mooresville  Iredell Vehicle Accident Congestion at 8/6 1:18 PM',
        "#Greece's tax revenues collapse as debt crisis continues via @guardian #bailout http://t.co/cJvbQXw83s ^mp",
        'Be not afraid of sudden fear neither of the desolation of the wicked when it cometh. For the Lord shall be thy... http://t.co/bP597YDs2b',
        'Storm batters Auckland and Northland: A violent overnight storm has battered Auckland and Northland uprooting... http://t.co/enrPGRgtTs',
        "Benzema increasingly looks to be a casualty of Benitez's new look squad. Arsenal bound? 50-50 chance I think",
        'Acquire your postexistence straight a elevation in addition

# Tokenization and Embeddings of the text

* Tokenization - A straight mapping from word or character or sub-word to a numerical value. Options:

 1) word-level tokenization

 2) Character-level tokenization

 3) Sub-word tokenization

* Embeddings - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector.

 1)Create a new embeeding using an embedding layer and an embedding representation will be learned during training.

 2) Reuse a pre-learned embbeding (like Word2vec embeddings, GloVe embeddings). Pre-trained embeddings have often been learned on large corpuses of text and thus have a good underlying representation of natural language.

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6


text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (with none it will be all of the different words in the text)
                                    standardize="lower_and_strip_punctuation", # how to process the text
                                    split="whitespace", # how to split tokens (word level tokenization)
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?


In [14]:
# Average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# customize text vectorization by imposing a max_vocab_length and  max lengh (seting it to the average lenth of tweets)
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[240,   3, 229,   4,  13, 724,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
ITS A TIE DYE EXPLOSION ON IG HELP ME. IM DROWNING IN TIE DYE      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  37,    3, 3059, 5287,  276,   11, 2691,  149,   32,   29,  450,
           4, 3059, 5287,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['minded', 'mindblowing', 'milne', 'milledgeville', 'millcityio']


In [21]:
# Creating an embbeding using a embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

In [22]:
# check an example: how a random sentence form the training set gets converted into a tensor containg an array of vectors
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
FAAN gives owners of abandoned aircraft evacuation ultimatum http://t.co/zZpojgngAJ via @dailytimesngr. They should probe them too!      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00330921,  0.04805082, -0.04638683, ...,  0.00557104,
         -0.0484992 ,  0.01777934],
        [ 0.04110588,  0.04038623, -0.03792178, ..., -0.02292033,
         -0.02309731, -0.03329172],
        [-0.00796854,  0.01231909,  0.03936482, ...,  0.02243838,
         -0.03152286,  0.03752461],
        ...,
        [ 0.01034749,  0.01760156,  0.04161297, ..., -0.00541585,
         -0.0094284 ,  0.03868672],
        [ 0.00799261, -0.02694448, -0.02190882, ...,  0.04008974,
          0.03780571,  0.04908166],
        [-0.03036931, -0.01917355,  0.00638828, ...,  0.04598478,
         -0.03835088,  0.03914836]]], dtype=float32)>

In [23]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.00330921,  0.04805082, -0.04638683, -0.01233204,  0.02307599,
        0.01834775, -0.02222123, -0.01184855,  0.0248148 , -0.00955632,
        0.02311074, -0.01385703, -0.02101959,  0.01447893, -0.04050596,
       -0.00683498, -0.01001896, -0.03988085, -0.02846166, -0.03219724,
        0.02216331, -0.02822023,  0.01456311,  0.03665078,  0.0177107 ,
       -0.02293092, -0.04545161, -0.04561467, -0.00640222, -0.00316773,
       -0.0422589 ,  0.0163944 , -0.00343693, -0.00652878, -0.03055564,
        0.03635204,  0.0241177 , -0.04250641,  0.03501439,  0.04809522,
        0.03829711,  0.02838215, -0.02580755,  0.01187426, -0.00406229,
        0.0479585 , -0.02921563,  0.01378151,  0.0131782 , -0.01609484,
        0.02554869, -0.01708747, -0.03288438,  0.03944934, -0.00801289,
       -0.01206994, -0.02658159, -0.02436755,  0.011634  , -0.03091947,
       -0.02641227, -0.01424453, -0.00643098,  0.02584404,  0.01930273,
        0.016384

# Modelling a test dataset

Looking at different models and comparing the results.



## Baseline Model - using Multinomial Naive Bayes algorithm

Multinomial Naive Bayes is suitable for classification with discrete features  (e.g., word counts for text classification)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf() - turns raw documents into a matrix of TF-IDF features. It accounts for the importance of words within each document relative to all other documents in the dataset
                    ("clf", MultinomialNB()) # model the text using Muktinomminal Naive Bayes
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.91%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [39]:
from sklearn.metrics import classification_report
model_0_results = classification_report(val_labels, baseline_preds)
print(model_0_results)

              precision    recall  f1-score   support

           0       0.76      0.94      0.84       858
           1       0.88      0.62      0.73       665

    accuracy                           0.80      1523
   macro avg       0.82      0.78      0.79      1523
weighted avg       0.81      0.80      0.79      1523



## Model 1 - Feed-forward NN (Simple Dense Model)

In [29]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [30]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [31]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [32]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 11s 45ms/step - loss: 0.6220 - accuracy: 0.6851 - val_loss: 0.5508 - val_accuracy: 0.7472
Epoch 2/5
191/191 [==============================] - 2s 12ms/step - loss: 0.4558 - accuracy: 0.8126 - val_loss: 0.4772 - val_accuracy: 0.7840
Epoch 3/5
191/191 [==============================] - 2s 8ms/step - loss: 0.3548 - accuracy: 0.8589 - val_loss: 0.4643 - val_accuracy: 0.7938
Epoch 4/5
191/191 [==============================] - 2s 13ms/step - loss: 0.2881 - accuracy: 0.8892 - val_loss: 0.4651 - val_accuracy: 0.7892
Epoch 5/5
191/191 [==============================] - 2s 9ms/step - loss: 0.2391 - accuracy: 0.9117 - val_loss: 0.4761 - val_accuracy: 0.7892


In [33]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

48/48 [==============================] - 0s 3ms/step - loss: 0.4761 - accuracy: 0.7892


[0.47614866495132446, 0.7892317771911621]

In [34]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.04783098,  0.00416015,  0.04901524, ..., -0.05485   ,
         -0.01932024,  0.00052411],
        [-0.01974662,  0.02969276,  0.04454971, ..., -0.0480215 ,
         -0.05094968, -0.0062669 ],
        [ 0.01684316,  0.02213525,  0.02770031, ...,  0.02680289,
         -0.01788897, -0.05193358],
        ...,
        [-0.04517051,  0.05431014,  0.04959549, ..., -0.02778558,
         -0.00288515, -0.07348221],
        [ 0.03799131, -0.02443846, -0.06192053, ..., -0.01006051,
          0.05140123,  0.00284522],
        [ 0.04833405, -0.1027751 , -0.10725585, ...,  0.04594267,
          0.05793495,  0.09392592]], dtype=float32)>]

In [35]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [37]:
# Make predictions (these will be in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

48/48 [==============================] - 0s 2ms/step


array([[0.41896284],
       [0.87560195],
       [0.9966336 ],
       [0.13793422],
       [0.10842457],
       [0.94615316],
       [0.9661112 ],
       [0.9925222 ],
       [0.93813205],
       [0.289486  ]], dtype=float32)

In [38]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions, by using tf.round any probability < 0.5 will result in target class 0 and => in 1)
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [40]:
model_1_results = classification_report(val_labels, model_1_preds)
print(model_1_results)

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       858
           1       0.80      0.69      0.74       665

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



## Model 2: LSTM model

In [41]:
# to make sure we will not used trained embeddings from the previous model a mode_2_embeddings will be created. Metrics on the training dataset are considerably higher than in the validation dataset

In [42]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
x = layers.Dense(64, activation="relu")(x) # adding a dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [43]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [45]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 12s 42ms/step - loss: 0.5144 - accuracy: 0.7376 - val_loss: 0.4579 - val_accuracy: 0.7859
Epoch 2/5
191/191 [==============================] - 3s 18ms/step - loss: 0.3058 - accuracy: 0.8744 - val_loss: 0.4951 - val_accuracy: 0.7912
Epoch 3/5
191/191 [==============================] - 3s 15ms/step - loss: 0.2036 - accuracy: 0.9251 - val_loss: 0.5950 - val_accuracy: 0.7748
Epoch 4/5
191/191 [==============================] - 3s 15ms/step - loss: 0.1340 - accuracy: 0.9509 - val_loss: 0.9052 - val_accuracy: 0.7413
Epoch 5/5
191/191 [==============================] - 3s 13ms/step - loss: 0.0954 - accuracy: 0.9640 - val_loss: 0.8549 - val_accuracy: 0.7754


In [ ]:
# both model_1 and model_2 are overfitting

In [46]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

48/48 [==============================] - 2s 4ms/step


((1523, 1),
 array([[0.04556294],
        [0.89523226],
        [0.9995184 ],
        [0.03035756],
        [0.00174429],
        [0.9990101 ],
        [0.9867116 ],
        [0.9999434 ],
        [0.99890554],
        [0.11433104]], dtype=float32))

In [47]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
model_2_results = classification_report(val_labels, model_2_preds)
print(model_2_results)

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       858
           1       0.80      0.65      0.72       665

    accuracy                           0.78      1523
   macro avg       0.78      0.76      0.77      1523
weighted avg       0.78      0.78      0.77      1523



## Model 3 - GRU

In [50]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
# again a new embbeding has to be created to avoid using pretrained wieghts
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)

x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [51]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                       

In [53]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 15s 56ms/step - loss: 0.5400 - accuracy: 0.7218 - val_loss: 0.4596 - val_accuracy: 0.7833
Epoch 2/5
191/191 [==============================] - 3s 13ms/step - loss: 0.3172 - accuracy: 0.8672 - val_loss: 0.4886 - val_accuracy: 0.7873
Epoch 3/5
191/191 [==============================] - 5s 24ms/step - loss: 0.2096 - accuracy: 0.9222 - val_loss: 0.5749 - val_accuracy: 0.7754
Epoch 4/5
191/191 [==============================] - 5s 24ms/step - loss: 0.1452 - accuracy: 0.9478 - val_loss: 0.6480 - val_accuracy: 0.7511
Epoch 5/5
191/191 [==============================] - 2s 10ms/step - loss: 0.1086 - accuracy: 0.9640 - val_loss: 0.6923 - val_accuracy: 0.7853


In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

48/48 [==============================] - 1s 3ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [55]:
model_3_results = classification_report(val_labels, model_3_preds)
print(model_3_results)

              precision    recall  f1-score   support

           0       0.77      0.88      0.82       858
           1       0.81      0.66      0.73       665

    accuracy                           0.79      1523
   macro avg       0.79      0.77      0.78      1523
weighted avg       0.79      0.79      0.78      1523



## Model 4: Bidirectonal RNN model

In [56]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [57]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [59]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 17s 57ms/step - loss: 0.5166 - accuracy: 0.7420 - val_loss: 0.4616 - val_accuracy: 0.7853
Epoch 2/5
191/191 [==============================] - 3s 16ms/step - loss: 0.3033 - accuracy: 0.8770 - val_loss: 0.4937 - val_accuracy: 0.7912
Epoch 3/5
191/191 [==============================] - 2s 11ms/step - loss: 0.1946 - accuracy: 0.9314 - val_loss: 0.5780 - val_accuracy: 0.7748
Epoch 4/5
191/191 [==============================] - 3s 13ms/step - loss: 0.1319 - accuracy: 0.9557 - val_loss: 0.6367 - val_accuracy: 0.7492
Epoch 5/5
191/191 [==============================] - 2s 10ms/step - loss: 0.0896 - accuracy: 0.9675 - val_loss: 0.7796 - val_accuracy: 0.7800


In [60]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

48/48 [==============================] - 1s 5ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [61]:
model_4_results = classification_report(val_labels, model_4_preds)
print(model_4_results)

              precision    recall  f1-score   support

           0       0.77      0.87      0.82       858
           1       0.80      0.66      0.72       665

    accuracy                           0.78      1523
   macro avg       0.78      0.77      0.77      1523
weighted avg       0.78      0.78      0.78      1523



## Model 5: Conv1D

In [63]:
# Test out the embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu") # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important features
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [65]:
# See the outputs of each layer
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.00113421, -0.04583002,  0.00187614, ..., -0.02568437,
          -0.01758539, -0.01753041],
         [-0.04538927,  0.06524515,  0.02988514, ..., -0.03371186,
          -0.02120916, -0.05951733],
         [-0.0394549 , -0.02281505, -0.00802676, ..., -0.00046356,
           0.03081732,  0.02348012],
         ...,
         [-0.04783098,  0.00416015,  0.04901524, ..., -0.05485   ,
          -0.01932024,  0.00052411],
         [-0.04783098,  0.00416015,  0.04901524, ..., -0.05485   ,
          -0.01932024,  0.00052411],
         [-0.04783098,  0.00416015,  0.04901524, ..., -0.05485   ,
          -0.01932024,  0.00052411]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.00000000e+00, 0.00000000e+00, 2.93501616e-02, 6.52210321e-04,
          0.00000000e+00, 0.00000000e+00, 1.67911276e-02, 0.00000000e+00,
          0.00000000e+00, 7.47672617e-02, 0.00000000e+00, 0.00000000e+00,
          0.

In [66]:
# put everything together

tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 64)             

In [67]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))


Epoch 1/5
191/191 [==============================] - 11s 40ms/step - loss: 0.5627 - accuracy: 0.7094 - val_loss: 0.4739 - val_accuracy: 0.7853
Epoch 2/5
191/191 [==============================] - 3s 16ms/step - loss: 0.3120 - accuracy: 0.8721 - val_loss: 0.4927 - val_accuracy: 0.7886
Epoch 3/5
191/191 [==============================] - 2s 12ms/step - loss: 0.1600 - accuracy: 0.9484 - val_loss: 0.5677 - val_accuracy: 0.7768
Epoch 4/5
191/191 [==============================] - 2s 10ms/step - loss: 0.0982 - accuracy: 0.9711 - val_loss: 0.6337 - val_accuracy: 0.7505
Epoch 5/5
191/191 [==============================] - 2s 9ms/step - loss: 0.0750 - accuracy: 0.9768 - val_loss: 0.6917 - val_accuracy: 0.7840


In [68]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

48/48 [==============================] - 0s 2ms/step


In [69]:
model_5_results = classification_report(val_labels, model_5_preds)
print(model_5_results)

              precision    recall  f1-score   support

           0       0.77      0.87      0.82       858
           1       0.80      0.67      0.73       665

    accuracy                           0.78      1523
   macro avg       0.79      0.77      0.78      1523
weighted avg       0.79      0.78      0.78      1523



## Using Pretrained embeddings (using the Universal Sentence Encoder from TensorFlow)


The embedding layer will be replaced by a pretrained embedding layer.

Rhe Universal Sentence encoder creates a sentensed-level embedding contrary to the used in models 0-5.

The output layer will be 512 dimensional vector for each sentence instead of a128 dimensional vector for each word

In [72]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

In [73]:
# passing a sentence through the Universal Sentence Encoder results in a vector with 512 dimensions
embed_samples[0].shape

TensorShape([512])

In [71]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

In [74]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [78]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
191/191 [==============================] - 3s 14ms/step - loss: 0.3777 - accuracy: 0.8327 - val_loss: 0.4375 - val_accuracy: 0.8089
Epoch 2/5
191/191 [==============================] - 3s 14ms/step - loss: 0.3718 - accuracy: 0.8340 - val_loss: 0.4351 - val_accuracy: 0.8122
Epoch 3/5
191/191 [==============================] - 4s 20ms/step - loss: 0.3661 - accuracy: 0.8381 - val_loss: 0.4391 - val_accuracy: 0.8096
Epoch 4/5
191/191 [==============================] - 3s 13ms/step - loss: 0.3597 - accuracy: 0.8424 - val_loss: 0.4372 - val_accuracy: 0.8135
Epoch 5/5
191/191 [==============================] - 2s 13ms/step - loss: 0.3549 - accuracy: 0.8427 - val_loss: 0.4387 - val_accuracy: 0.8148


In [76]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

48/48 [==============================] - 2s 21ms/step


In [77]:
model_6_results = classification_report(val_labels, model_6_preds)
print(model_6_results)

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       858
           1       0.80      0.75      0.77       665

    accuracy                           0.81      1523
   macro avg       0.81      0.80      0.80      1523
weighted avg       0.81      0.81      0.81      1523



This last model the training and validation metrics are a lot closer (low or no overfitting) and the overall metrics have improved

# Making predictions on the test dataset

In [88]:
# Making predictions on the test dataset
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6.predict([test_sample])) # has to be list
  pred = tf.round(pred_prob)
  print(f"Pred: {int(pred)}, Prob: {pred_prob}")
  print(f"Text:\n{test_sample}\n")
  print("----\n")

1/1 [==============================] - 0s 92ms/step
Pred: 1, Prob: 0.9154991507530212
Text:
30 Years After the Chernobyl Nuclear Disaster via @fubiz http://t.co/N4B59iqm94

----

1/1 [==============================] - 0s 62ms/step
Pred: 0, Prob: 0.11785832792520523
Text:
VAL JUST DIED AND IM ABSOLUTELY DEVASTATED #emmerdale

----

1/1 [==============================] - 0s 62ms/step
Pred: 0, Prob: 0.11346697807312012
Text:
@cierranb5678 I was just there and my dad told me he would never let me get an Avalanche

----

1/1 [==============================] - 0s 67ms/step
Pred: 1, Prob: 0.9175835847854614
Text:
Kurdish Militants Target Turkish Military In Suicide Attack - Huffington Post http://t.co/03bJm4ORoW

----

1/1 [==============================] - 0s 55ms/step
Pred: 1, Prob: 0.988174557685852
Text:
Debris Found in Indian Ocean could be Malaysia Flight 370! http://t.co/VUoJPrKUAX

----

1/1 [==============================] - 0s 63ms/step
Pred: 0, Prob: 0.17485399544239044
Text:
#hot 